In [ ]:
from pyonepassword import OP
ST_CREDENTIALS = OP().item_get('justrealtrading.com (ivanrdgc@gmail.com)')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

from tqdm.auto import tqdm
import requests
import pathlib

def download_file(url: str, destination: pathlib.Path):
    # Streaming, so we can iterate over the response.
    response = requests.get(url, stream=True)
    total_size_in_bytes= int(response.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True, leave=False)
    with open(destination, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()
    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print(f"ERROR, something went wrong {url} {destination}")

def get_st_html(username: str, password: str, base_path: pathlib.Path = pathlib.Path('out'), headless: bool = True) -> str:
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 90)

    driver.get('https://secretfloor.justrealtrading.com/Acceso/')
    elem = driver.find_element(By.XPATH, '//*[@name="email"]')
    elem.send_keys(username)
    elem = driver.find_element(By.XPATH, '//*[@name="password"]')
    elem.send_keys(password)
    elem.send_keys(Keys.RETURN)

    wait.until(EC.presence_of_element_located((By.XPATH, '//a[@title="Smart Trades"]'))).click()
    wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')

    data = []
    for el in tqdm(driver.find_elements(By.XPATH, '//*[@id="post-12770"]/*[@class="art-postcontent clearfix"]/p/strong/..|//*[@id="post-12770"]/*[@class="art-postcontent clearfix"]/strong')[::-1]):
        ctel = el.find_element(By.XPATH, 'following-sibling::*[@class="su-tabs su-tabs-style-default"]')
        tabs = ctel.find_elements(By.XPATH, '*[@class="su-tabs-nav"]/span')
        w_data = []
        for tab in tabs:
            tab.click()
            wait.until(lambda driver: tab.get_attribute('class') == 'su-tabs-current')
            cntent = ctel.find_element(By.XPATH, '*[@class="su-tabs-panes"]/*[@class="su-tabs-pane su-clearfix"][@style="display: block;"]')
            try:
                iframe = cntent.find_element(By.XPATH, '*/iframe')#.get_attribute('src')
                driver.switch_to.frame(iframe)
                wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
                wait.until(lambda driver: driver.execute_script('return ("playerConfig" in window) ? "true" : "false"') == 'true')
                iframe = driver.execute_script('return playerConfig.request.files.progressive.find(p => p.quality=="1080p").url')
                driver.switch_to.default_content()
            except NoSuchElementException:
                iframe = None
            w_data.append([tab.text.strip(), cntent.text.strip(), iframe])
            if iframe is not None:
                base_path.mkdir(exist_ok=True)
                dl_path = base_path.joinpath(iframe.rsplit('/',1)[-1])
                if not dl_path.exists():
                    download_file(iframe, dl_path)

        data.append([el.text.strip(), w_data])

    driver.quit()
    return data

data = get_st_html(ST_CREDENTIALS.username, ST_CREDENTIALS.password, headless=False)